# Sources

* https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
* https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
* https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/
* https://machinelearningmastery.com/visualize-time-series-residual-forecast-errors-with-python/

# Initial Set-Up

In [1]:
# Install Dependencies
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, TimeDistributed
from numpy import concatenate
from math import sqrt
from statsmodels.graphics.gofplots import qqplot
from pandas.plotting import autocorrelation_plot
from tensorflow.keras.layers import Conv1D, MaxPooling1D

In [2]:
# import NYT dataset
df=pd.read_csv("Potential Data Sources/NYT- Cty Cases Deaths Thru Oct2/us-counties.csv", parse_dates=['date'])
# limit dataset to texas
Texas=df.loc[(df['state']=='Texas')].dropna(how='any', axis=0)
Texas2=Texas.drop(['state'], axis=1)

# Functions for Model

In [3]:
# define function to create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [4]:
# define function to convert series to supervised learning
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [5]:
# define function to scale train and test data to [-1, 1] to mimic tanh default activation function for LSTMs
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [6]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [7]:
# define function to fit an LSTM network to training data - 
# do not want to shuffle as want the network to build up as it learns across the sequence of observations
def fit_cnn_lstm(train, batch_size, nb_epoch, neurons):
    # choose a number of time steps
    n_steps = 4
    # split into samples
    X, y = split_sequence(train[:, 0:-1], n_steps)
    # reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
    n_features = 1
    n_seq = 2
    n_steps = 2
    X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
    
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()

    return model

# Case Data Loop through Counties

In [8]:
# convert fips to integer
Texas2['fips'] = Texas2['fips'].astype(int)

In [9]:
# drop counties that have 0 cases before 7/31
Texas2 = Texas2[(Texas2.fips != 48033) & (Texas2.fips != 48431)]

In [10]:
# Form unique list of fips
TX_fips=Texas2['fips'].unique().tolist()
len(TX_fips)

250

In [11]:
# Form unique list of counties
TX_counties=Texas2['county'].unique().tolist()
len(TX_counties)

250

In [12]:
# loop through counties to obtain case data for every county

case_results = []
lstm_case_rmse = []

i = 0 
while i < len(TX_counties):
    county=str(TX_counties[i])
    fips=TX_fips[i]
    print(county)
    print(fips)
    # limit dataset to certain County
    countyData = Texas2.loc[(Texas2['county']==county)].reset_index(drop=True)
    County_Cases=countyData.drop(['deaths', 'county', 'fips'], axis=1)
    County_Cases=County_Cases.set_index('date')
    County_Cases=County_Cases.fillna(0)
  
    print(f' Currently processing {county}, number {i+1} of {len(TX_counties)} counties')

    #  test train split
    start_date = countyData.iloc[0,0]
    train_days = pd.to_datetime('2020-08-01') - start_date
    train_number = train_days.days
    # transform data to be stationary
    raw_values = County_Cases.values
    diff_values = difference(raw_values, 1)
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(diff_values, 1)
    supervised_values = supervised.values
    # split into train and test sets
    train = supervised_values[:train_number, :]
    test = supervised_values[train_number:, :]
    
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    
    # fit the model using batch_size =1, epochs=200, neurons=4
    lstm_model = fit_cnn_lstm(train_scaled, 1, 200, 4)

    # Predict values on test set
    # choose a number of time steps
    n_steps = 4
    # split into samples
    X, y = split_sequence(test_scaled[:, 0:-1], n_steps)
    # demonstrate prediction
    n_features = 1
    n_seq = 2
    n_steps = 2
    # X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
    x_input = X.reshape(X.shape[0], n_seq, n_steps, n_features)
    yhat = lstm_model.predict(x_input, verbose=0)
    
    # Form array with test data and predicted value to prepare for scalar invert
    yhat_array = np.hstack((test_scaled[4:, 0:-1],yhat))
    
    # Invert scaled test data and predicted data
    inverted = scaler.inverse_transform(yhat_array)
    
    # Convert results into dataframe to store only predicted values
    inverted_df = pd.DataFrame(data=inverted, columns=["lstm_cases_expected", "lstm_cases_predicted"])
    predicted_df = inverted_df.drop(columns='lstm_cases_expected')
    
    # Get last day of training dataset cumulative cases to undifference
    cum_case = raw_values[train_number+3]
    
    predictions = predicted_df.cumsum()+cum_case
    
    # Calculate rmse and store
    rmse = sqrt(mean_squared_error(inverted_df['lstm_cases_expected'], inverted_df['lstm_cases_predicted']))
    lstm_case_rmse.append(rmse)
    
    # Calculate residuals
    residuals = inverted_df['lstm_cases_expected'] - inverted_df['lstm_cases_predicted']
    residuals_df = pd.DataFrame(data=residuals, columns=["lstm_cases_residuals"])
    
    # Put everything together in dataframe
    dates = (countyData.loc[train_number+5:]['date']).reset_index(drop=True)
    results_df = predictions.join(residuals_df)
    results_df['fips_code']= fips
    results_df['date']= dates
    case_results.append(results_df)

    i = i + 1
    print("Done")

Bexar
48029
 Currently processing Bexar, number 1 of 250 counties
Done
Fort Bend
48157
 Currently processing Fort Bend, number 2 of 250 counties
Done
Harris
48201
 Currently processing Harris, number 3 of 250 counties
Done
Collin
48085
 Currently processing Collin, number 4 of 250 counties
Done
Dallas
48113
 Currently processing Dallas, number 5 of 250 counties
Done
Montgomery
48339
 Currently processing Montgomery, number 6 of 250 counties
Done
Tarrant
48439
 Currently processing Tarrant, number 7 of 250 counties
Done
Gregg
48183
 Currently processing Gregg, number 8 of 250 counties
Done
Bell
48027
 Currently processing Bell, number 9 of 250 counties
Done
El Paso
48141
 Currently processing El Paso, number 10 of 250 counties
Done
Galveston
48167
 Currently processing Galveston, number 11 of 250 counties
Done
Smith
48423
 Currently processing Smith, number 12 of 250 counties
Done
Travis
48453
 Currently processing Travis, number 13 of 250 counties
Done
Brazoria
48039
 Currently process

Done
Hays
48209
 Currently processing Hays, number 15 of 250 counties
Done
Denton
48121
 Currently processing Denton, number 16 of 250 counties
Done
Bowie
48037
 Currently processing Bowie, number 17 of 250 counties
Done
Matagorda
48321
 Currently processing Matagorda, number 18 of 250 counties
Done
Brazos
48041
 Currently processing Brazos, number 19 of 250 counties
Done
Ellis
48139
 Currently processing Ellis, number 20 of 250 counties
Done
Lubbock
48303
 Currently processing Lubbock, number 21 of 250 counties
Done
Lavaca
48285
 Currently processing Lavaca, number 22 of 250 counties
Done
McLennan
48309
 Currently processing McLennan, number 23 of 250 counties
Done
Medina
48325
 Currently processing Medina, number 24 of 250 counties


Done
Rusk
48401
 Currently processing Rusk, number 25 of 250 counties
Done
Webb
48479
 Currently processing Webb, number 26 of 250 counties
Done
Wichita
48485
 Currently processing Wichita, number 27 of 250 counties
Done
Williamson
48491
 Currently processing Williamson, number 28 of 250 counties
Done
Cameron
48061
 Currently processing Cameron, number 29 of 250 counties
Done
Crane
48103
 Currently processing Crane, number 30 of 250 counties
Done
DeWitt
48123
 Currently processing DeWitt, number 31 of 250 counties
Done
Johnson
48251
 Currently processing Johnson, number 32 of 250 counties
Done
Van Zandt
48467
 Currently processing Van Zandt, number 33 of 250 counties
Done
Brown
48049
 Currently processing Brown, number 34 of 250 counties


Done
Cass
48067
 Currently processing Cass, number 35 of 250 counties
Done
Fannin
48147
 Currently processing Fannin, number 36 of 250 counties
Done
Hockley
48219
 Currently processing Hockley, number 37 of 250 counties
Done
Upshur
48459
 Currently processing Upshur, number 38 of 250 counties
Done
Bastrop
48021
 Currently processing Bastrop, number 39 of 250 counties
Done
Castro
48069
 Currently processing Castro, number 40 of 250 counties
Done
Deaf Smith
48117
 Currently processing Deaf Smith, number 41 of 250 counties
Done
Midland
48329
 Currently processing Midland, number 42 of 250 counties
Done
Oldham
48359
 Currently processing Oldham, number 43 of 250 counties
Done
Potter
48375
 Currently processing Potter, number 44 of 250 counties


Done
Morris
48343
 Currently processing Morris, number 45 of 250 counties
Done
Chambers
48071
 Currently processing Chambers, number 46 of 250 counties
Done
Comal
48091
 Currently processing Comal, number 47 of 250 counties
Done
Falls
48145
 Currently processing Falls, number 48 of 250 counties
Done
Grayson
48181
 Currently processing Grayson, number 49 of 250 counties
Done
Hidalgo
48215
 Currently processing Hidalgo, number 50 of 250 counties
Done
Jefferson
48245
 Currently processing Jefferson, number 51 of 250 counties
Done
Nueces
48355
 Currently processing Nueces, number 52 of 250 counties
Done
Parker
48367
 Currently processing Parker, number 53 of 250 counties
Done
Victoria
48469
 Currently processing Victoria, number 54 of 250 counties


...

Done
Cottle
48101
 Currently processing Cottle, number 205 of 250 counties
Done
Lipscomb
48295
 Currently processing Lipscomb, number 206 of 250 counties
Done
Runnels
48399
 Currently processing Runnels, number 207 of 250 counties
Done
Garza
48169
 Currently processing Garza, number 208 of 250 counties
Done
Glasscock
48173
 Currently processing Glasscock, number 209 of 250 counties
Done
Brewster
48043
 Currently processing Brewster, number 210 of 250 counties
Done
Briscoe
48045
 Currently processing Briscoe, number 211 of 250 counties
Done
Coleman
48083
 Currently processing Coleman, number 212 of 250 counties
Done
Refugio
48391
 Currently processing Refugio, number 213 of 250 counties
Done
Bailey
48017
 Currently processing Bailey, number 214 of 250 counties


Done
Coke
48081
 Currently processing Coke, number 215 of 250 counties
Done
Collingsworth
48087
 Currently processing Collingsworth, number 216 of 250 counties
Done
Shackelford
48417
 Currently processing Shackelford, number 217 of 250 counties
Done
Kimble
48267
 Currently processing Kimble, number 218 of 250 counties
Done
Reeves
48389
 Currently processing Reeves, number 219 of 250 counties
Done
Archer
48009
 Currently processing Archer, number 220 of 250 counties
Done
Fisher
48151
 Currently processing Fisher, number 221 of 250 counties
Done
Haskell
48207
 Currently processing Haskell, number 222 of 250 counties
Done
Mills
48333
 Currently processing Mills, number 223 of 250 counties
Done
San Saba
48411
 Currently processing San Saba, number 224 of 250 counties


Done
Hall
48191
 Currently processing Hall, number 225 of 250 counties
Done
Hardeman
48197
 Currently processing Hardeman, number 226 of 250 counties
Done
Ward
48475
 Currently processing Ward, number 227 of 250 counties
Done
Presidio
48377
 Currently processing Presidio, number 228 of 250 counties
Done
Hudspeth
48229
 Currently processing Hudspeth, number 229 of 250 counties
Done
Reagan
48383
 Currently processing Reagan, number 230 of 250 counties
Done
Kinney
48271
 Currently processing Kinney, number 231 of 250 counties
Done
Irion
48235
 Currently processing Irion, number 232 of 250 counties
Done
Baylor
48023
 Currently processing Baylor, number 233 of 250 counties
Done
Crockett
48105
 Currently processing Crockett, number 234 of 250 counties


Done
Real
48385
 Currently processing Real, number 235 of 250 counties
Done
Somervell
48425
 Currently processing Somervell, number 236 of 250 counties
Done
Schleicher
48413
 Currently processing Schleicher, number 237 of 250 counties
Done
Sutton
48435
 Currently processing Sutton, number 238 of 250 counties
Done
Upton
48461
 Currently processing Upton, number 239 of 250 counties
Done
Culberson
48109
 Currently processing Culberson, number 240 of 250 counties
Done
Menard
48327
 Currently processing Menard, number 241 of 250 counties
Done
Terrell
48443
 Currently processing Terrell, number 242 of 250 counties
Done
Edwards
48137
 Currently processing Edwards, number 243 of 250 counties
Done
Jeff Davis
48243
 Currently processing Jeff Davis, number 244 of 250 counties


Done
Kenedy
48261
 Currently processing Kenedy, number 245 of 250 counties
Done
Throckmorton
48447
 Currently processing Throckmorton, number 246 of 250 counties
Done
Foard
48155
 Currently processing Foard, number 247 of 250 counties
Done
Kent
48263
 Currently processing Kent, number 248 of 250 counties
Done
McMullen
48311
 Currently processing McMullen, number 249 of 250 counties
Done
Stonewall
48433
 Currently processing Stonewall, number 250 of 250 counties
Done


In [13]:
lstm_case_rmse_df = pd.DataFrame(data=lstm_case_rmse, columns=["lstm_case_rmse"])
lstm_case_rmse_df

,lstm_case_rmse
0,674.947763
1,446.791884
2,3934.812550
3,529.042856
4,4042.418094
...,...
245,0.255303
246,1.764660
247,0.380984
248,0.799582


In [14]:
lstm_case_rmse_df['fips_code']=TX_fips[:len(lstm_case_rmse_df)]
lstm_case_rmse_df

,lstm_case_rmse,fips_code
0,674.947763,48029
1,446.791884,48157
2,3934.812550,48201
3,529.042856,48085
4,4042.418094,48113
...,...,...
245,0.255303,48447
246,1.764660,48155
247,0.380984,48263
248,0.799582,48311


In [34]:
lstm_case_rmse_df.set_index('fips_code', inplace = True)

In [55]:
lstm_case_data = pd.concat(case_results,ignore_index=True)
lstm_case_data

,lstm_cases_predicted,lstm_cases_residuals,fips_code,date
0,41362.054108,263.945892,48029,2020-08-06
1,41579.527377,107.526731,48029,2020-08-07
2,42228.470455,-288.943077,48029,2020-08-08
3,42958.284765,-497.814310,48029,2020-08-09
4,43774.518453,-564.233688,48029,2020-08-10
...,...,...,...,...
14745,2.717025,0.000000,48433,2020-09-29
14746,2.717025,0.000000,48433,2020-09-30
14747,2.717025,0.000000,48433,2020-10-01
14748,2.717025,2.000000,48433,2020-10-02


In [56]:
lstm_case_data.set_index(['date','fips_code'], inplace = True)

In [57]:
lstm_case_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 14750 entries, (Timestamp('2020-08-06 00:00:00'), 48029) to (Timestamp('2020-10-03 00:00:00'), 48433)
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   lstm_cases_predicted  14750 non-null  float64
 1   lstm_cases_residuals  14750 non-null  float64
dtypes: float64(2)
memory usage: 288.7 KB


# Death Data Loop through Counties

In [20]:
Texas2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44385 entries, 143 to 595652
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    44385 non-null  datetime64[ns]
 1   county  44385 non-null  object        
 2   fips    44385 non-null  int32         
 3   cases   44385 non-null  int64         
 4   deaths  44385 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(2), object(1)
memory usage: 1.9+ MB


In [21]:
# drop counties that have 0 deaths before 7/31
Texas2 = Texas2[(Texas2.fips != 48009) & (Texas2.fips != 48011) & (Texas2.fips != 48023) & (Texas2.fips != 48065) & 
                (Texas2.fips != 48075) & (Texas2.fips != 48079) & (Texas2.fips != 48083) & (Texas2.fips != 48087) & 
                (Texas2.fips != 48095) & (Texas2.fips != 48119) & (Texas2.fips != 48129) & (Texas2.fips != 48137) & 
                (Texas2.fips != 48155) & (Texas2.fips != 48161) & (Texas2.fips != 48165) & (Texas2.fips != 48173) & 
                (Texas2.fips != 48197) & (Texas2.fips != 48207) & (Texas2.fips != 48211) & (Texas2.fips != 48235) & 
                (Texas2.fips != 48237) & (Texas2.fips != 48243) & (Texas2.fips != 48253) & (Texas2.fips != 48263) & 
                (Texas2.fips != 48267) & (Texas2.fips != 48271) & (Texas2.fips != 48275) & (Texas2.fips != 48295) & 
                (Texas2.fips != 48311) & (Texas2.fips != 48313) & (Texas2.fips != 48319) & (Texas2.fips != 48327) & 
                (Texas2.fips != 48333) & (Texas2.fips != 48335) & (Texas2.fips != 48345) & (Texas2.fips != 48377) & 
                (Texas2.fips != 48385) & (Texas2.fips != 48391) & (Texas2.fips != 48393) & (Texas2.fips != 48411) & 
                (Texas2.fips != 48417) & (Texas2.fips != 48421) & (Texas2.fips != 48425) & (Texas2.fips != 48433) & 
                (Texas2.fips != 48443) & (Texas2.fips != 48447) & (Texas2.fips != 48483)]

In [22]:
Texas2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37523 entries, 143 to 595652
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    37523 non-null  datetime64[ns]
 1   county  37523 non-null  object        
 2   fips    37523 non-null  int32         
 3   cases   37523 non-null  int64         
 4   deaths  37523 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(2), object(1)
memory usage: 1.6+ MB


In [23]:
# Form unique list of fips
TX_fips=Texas2['fips'].unique().tolist()
len(TX_fips)

203

In [24]:
# Form unique list of counties
TX_counties=Texas2['county'].unique().tolist()
len(TX_counties)

203

In [25]:
# loop through counties to obtain death data for every county

death_results = []
lstm_death_rmse = []

i = 0 
while i < len(TX_counties):
    county=str(TX_counties[i])
    fips=TX_fips[i]
    print(county)
    print(fips)
    # limit dataset to certain County
    countyData = Texas2.loc[(Texas2['county']==county)].reset_index(drop=True)
    County_Deaths=countyData.drop(['cases', 'county', 'fips'], axis=1)
    County_Deaths=County_Deaths.set_index('date')
    County_Deaths=County_Deaths.fillna(0)
  
    print(f' Currently processing {county}, number {i+1} of {len(TX_counties)} counties')

    #  test train split
    start_date = countyData.iloc[0,0]
    train_days = pd.to_datetime('2020-08-01') - start_date
    train_number = train_days.days
    # transform data to be stationary
    raw_values = County_Deaths.values
    diff_values = difference(raw_values, 1)
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(diff_values, 1)
    supervised_values = supervised.values
    # split into train and test sets
    train = supervised_values[:train_number, :]
    test = supervised_values[train_number:, :]
    
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    
    # fit the model using batch_size =1, epochs=200, neurons=4
    lstm_model = fit_cnn_lstm(train_scaled, 1, 200, 4)

    # Predict values on test set
    # choose a number of time steps
    n_steps = 4
    # split into samples
    X, y = split_sequence(test_scaled[:, 0:-1], n_steps)
    # demonstrate prediction
    n_features = 1
    n_seq = 2
    n_steps = 2
    # X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
    x_input = X.reshape(X.shape[0], n_seq, n_steps, n_features)
    yhat = lstm_model.predict(x_input, verbose=0)
    
    # Form array with test data and predicted value to prepare for scalar invert
    yhat_array = np.hstack((test_scaled[4:, 0:-1],yhat))
    
    # Invert scaled test data and predicted data
    inverted = scaler.inverse_transform(yhat_array)
    
    # Convert results into dataframe to store only predicted values
    inverted_df = pd.DataFrame(data=inverted, columns=["lstm_deaths_expected", "lstm_deaths_predicted"])
    predicted_df = inverted_df.drop(columns='lstm_deaths_expected')
    
    # Get last day of training dataset cumulative cases to undifference
    cum_death = raw_values[train_number+3]
    
    predictions = predicted_df.cumsum()+cum_death
    
    # Calculate rmse and store
    rmse = sqrt(mean_squared_error(inverted_df['lstm_deaths_expected'], inverted_df['lstm_deaths_predicted']))
    lstm_death_rmse.append(rmse)
    
    # Calculate residuals
    residuals = inverted_df['lstm_deaths_expected'] - inverted_df['lstm_deaths_predicted']
    residuals_df = pd.DataFrame(data=residuals, columns=["lstm_deaths_residuals"])
    
    # Put everything together in dataframe
    dates = (countyData.loc[train_number+5:]['date']).reset_index(drop=True)
    results_df = predictions.join(residuals_df)
    results_df['fips_code']= fips
    results_df['date']= dates
    death_results.append(results_df)

    i = i + 1
    print("Done")

Bexar
48029
 Currently processing Bexar, number 1 of 203 counties
Done
Fort Bend
48157
 Currently processing Fort Bend, number 2 of 203 counties
Done
Harris
48201
 Currently processing Harris, number 3 of 203 counties
Done
Collin
48085
 Currently processing Collin, number 4 of 203 counties
Done
Dallas
48113
 Currently processing Dallas, number 5 of 203 counties
Done
Montgomery
48339
 Currently processing Montgomery, number 6 of 203 counties
Done
Tarrant
48439
 Currently processing Tarrant, number 7 of 203 counties
Done
Gregg
48183
 Currently processing Gregg, number 8 of 203 counties
Done
Bell
48027
 Currently processing Bell, number 9 of 203 counties
Done
El Paso
48141
 Currently processing El Paso, number 10 of 203 counties


Done
Galveston
48167
 Currently processing Galveston, number 11 of 203 counties
Done
Smith
48423
 Currently processing Smith, number 12 of 203 counties
Done
Travis
48453
 Currently processing Travis, number 13 of 203 counties
Done
Brazoria
48039
 Currently processing Brazoria, number 14 of 203 counties
Done
Hays
48209
 Currently processing Hays, number 15 of 203 counties
Done
Denton
48121
 Currently processing Denton, number 16 of 203 counties
Done
Bowie
48037
 Currently processing Bowie, number 17 of 203 counties
Done
Matagorda
48321
 Currently processing Matagorda, number 18 of 203 counties
Done
Brazos
48041
 Currently processing Brazos, number 19 of 203 counties
Done
Ellis
48139
 Currently processing Ellis, number 20 of 203 counties


Done
Lubbock
48303
 Currently processing Lubbock, number 21 of 203 counties
Done
Lavaca
48285
 Currently processing Lavaca, number 22 of 203 counties
Done
McLennan
48309
 Currently processing McLennan, number 23 of 203 counties
Done
Medina
48325
 Currently processing Medina, number 24 of 203 counties
Done
Rusk
48401
 Currently processing Rusk, number 25 of 203 counties
Done
Webb
48479
 Currently processing Webb, number 26 of 203 counties
Done
Wichita
48485
 Currently processing Wichita, number 27 of 203 counties
Done
Williamson
48491
 Currently processing Williamson, number 28 of 203 counties
Done
Cameron
48061
 Currently processing Cameron, number 29 of 203 counties
Done
Crane
48103
 Currently processing Crane, number 30 of 203 counties


Done
DeWitt
48123
 Currently processing DeWitt, number 31 of 203 counties
Done
Johnson
48251
 Currently processing Johnson, number 32 of 203 counties
Done
Van Zandt
48467
 Currently processing Van Zandt, number 33 of 203 counties
Done
Brown
48049
 Currently processing Brown, number 34 of 203 counties
Done
Cass
48067
 Currently processing Cass, number 35 of 203 counties
Done
Fannin
48147
 Currently processing Fannin, number 36 of 203 counties
Done
Hockley
48219
 Currently processing Hockley, number 37 of 203 counties
Done
Upshur
48459
 Currently processing Upshur, number 38 of 203 counties
Done
Bastrop
48021
 Currently processing Bastrop, number 39 of 203 counties
Done
Castro
48069
 Currently processing Castro, number 40 of 203 counties


Done
Deaf Smith
48117
 Currently processing Deaf Smith, number 41 of 203 counties
Done
Midland
48329
 Currently processing Midland, number 42 of 203 counties
Done
Oldham
48359
 Currently processing Oldham, number 43 of 203 counties
Done
Potter
48375
 Currently processing Potter, number 44 of 203 counties
Done
Morris
48343
 Currently processing Morris, number 45 of 203 counties
Done
Chambers
48071
 Currently processing Chambers, number 46 of 203 counties
Done
Comal
48091
 Currently processing Comal, number 47 of 203 counties
Done
Falls
48145
 Currently processing Falls, number 48 of 203 counties
Done
Grayson
48181
 Currently processing Grayson, number 49 of 203 counties
Done
Hidalgo
48215
 Currently processing Hidalgo, number 50 of 203 counties


...

Done
McCulloch
48307
 Currently processing McCulloch, number 161 of 203 counties
Done
Sabine
48403
 Currently processing Sabine, number 162 of 203 counties
Done
Scurry
48415
 Currently processing Scurry, number 163 of 203 counties
Done
Cooke
48097
 Currently processing Cooke, number 164 of 203 counties
Done
Howard
48227
 Currently processing Howard, number 165 of 203 counties
Done
Rains
48379
 Currently processing Rains, number 166 of 203 counties
Done
Stephens
48429
 Currently processing Stephens, number 167 of 203 counties
Done
Winkler
48495
 Currently processing Winkler, number 168 of 203 counties
Done
Dimmit
48127
 Currently processing Dimmit, number 169 of 203 counties
Done
Duval
48131
 Currently processing Duval, number 170 of 203 counties


Done
Wilbarger
48487
 Currently processing Wilbarger, number 171 of 203 counties
Done
Bosque
48035
 Currently processing Bosque, number 172 of 203 counties
Done
La Salle
48283
 Currently processing La Salle, number 173 of 203 counties
Done
Marion
48315
 Currently processing Marion, number 174 of 203 counties
Done
Red River
48387
 Currently processing Red River, number 175 of 203 counties
Done
Zavala
48507
 Currently processing Zavala, number 176 of 203 counties
Done
Yoakum
48501
 Currently processing Yoakum, number 177 of 203 counties
Done
Houston
48225
 Currently processing Houston, number 178 of 203 counties
Done
Parmer
48369
 Currently processing Parmer, number 179 of 203 counties
Done
Hartley
48205
 Currently processing Hartley, number 180 of 203 counties


Done
Brooks
48047
 Currently processing Brooks, number 181 of 203 counties
Done
Jim Hogg
48247
 Currently processing Jim Hogg, number 182 of 203 counties
Done
Nolan
48353
 Currently processing Nolan, number 183 of 203 counties
Done
Ochiltree
48357
 Currently processing Ochiltree, number 184 of 203 counties
Done
Cottle
48101
 Currently processing Cottle, number 185 of 203 counties
Done
Runnels
48399
 Currently processing Runnels, number 186 of 203 counties
Done
Garza
48169
 Currently processing Garza, number 187 of 203 counties
Done
Brewster
48043
 Currently processing Brewster, number 188 of 203 counties
Done
Briscoe
48045
 Currently processing Briscoe, number 189 of 203 counties
Done
Bailey
48017
 Currently processing Bailey, number 190 of 203 counties


Done
Coke
48081
 Currently processing Coke, number 191 of 203 counties
Done
Reeves
48389
 Currently processing Reeves, number 192 of 203 counties
Done
Fisher
48151
 Currently processing Fisher, number 193 of 203 counties
Done
Hall
48191
 Currently processing Hall, number 194 of 203 counties
Done
Ward
48475
 Currently processing Ward, number 195 of 203 counties
Done
Hudspeth
48229
 Currently processing Hudspeth, number 196 of 203 counties
Done
Reagan
48383
 Currently processing Reagan, number 197 of 203 counties
Done
Crockett
48105
 Currently processing Crockett, number 198 of 203 counties
Done
Schleicher
48413
 Currently processing Schleicher, number 199 of 203 counties
Done
Sutton
48435
 Currently processing Sutton, number 200 of 203 counties


Done
Upton
48461
 Currently processing Upton, number 201 of 203 counties
Done
Culberson
48109
 Currently processing Culberson, number 202 of 203 counties
Done
Kenedy
48261
 Currently processing Kenedy, number 203 of 203 counties
Done


In [26]:
lstm_death_rmse_df = pd.DataFrame(data=lstm_death_rmse, columns=["lstm_death_rmse"])
lstm_death_rmse_df

,lstm_death_rmse
0,22.866424
1,3.571553
2,18.689728
3,1.846722
4,7.510754
...,...
198,0.129888
199,0.180644
200,0.129737
201,0.075981


In [27]:
lstm_death_rmse_df['fips_code']=TX_fips[:len(lstm_death_rmse_df)]
lstm_death_rmse_df

,lstm_death_rmse,fips_code
0,22.866424,48029
1,3.571553,48157
2,18.689728,48201
3,1.846722,48085
4,7.510754,48113
...,...,...
198,0.129888,48413
199,0.180644,48435
200,0.129737,48461
201,0.075981,48109


In [35]:
lstm_death_rmse_df.set_index('fips_code', inplace = True)

In [37]:
lstm_rmse_df = lstm_case_rmse_df.join(lstm_death_rmse_df, how='outer')
lstm_rmse_df

,lstm_case_rmse,lstm_death_rmse
fips_code,,
48001,201.006246,0.920685
48003,7.557863,0.276380
48005,29.049744,1.112880
48007,15.764733,0.364374
48009,5.178137,NaN
...,...,...
48499,7.944641,0.655776
48501,4.546793,0.220512
48503,22.341789,0.181215


In [43]:
lstm_rmse_df.to_csv('Database_Files/lstm_rmse.csv', index=True)

In [52]:
lstm_death_data = pd.concat(death_results,ignore_index=True)
lstm_death_data

,lstm_deaths_predicted,lstm_deaths_residuals,fips_code,date
0,641.585294,31.414706,48029,2020-08-06
1,630.076782,43.508512,48029,2020-08-07
2,645.043091,19.033691,48029,2020-08-08
3,676.483698,2.559393,48029,2020-08-09
4,709.281293,-23.797596,48029,2020-08-10
...,...,...,...,...
11972,3.187040,-0.041958,48261,2020-09-29
11973,3.228997,-0.041958,48261,2020-09-30
11974,3.270955,-0.041958,48261,2020-10-01
11975,3.312912,-0.041958,48261,2020-10-02


In [53]:
lstm_death_data.set_index(['date','fips_code'], inplace = True)

In [54]:
lstm_death_data

,,lstm_deaths_predicted,lstm_deaths_residuals
date,fips_code,,
2020-08-06,48029,641.585294,31.414706
2020-08-07,48029,630.076782,43.508512
2020-08-08,48029,645.043091,19.033691
2020-08-09,48029,676.483698,2.559393
2020-08-10,48029,709.281293,-23.797596
...,...,...,...
2020-09-29,48261,3.187040,-0.041958
2020-09-30,48261,3.228997,-0.041958
2020-10-01,48261,3.270955,-0.041958


In [58]:
lstm_data = lstm_case_data.join(lstm_death_data, how='outer')
lstm_data

lstm_cases_predicted  lstm_cases_residuals  \
date       fips_code                                               
2020-08-06 48001               2367.558983            -12.558983   
           48003                273.651881             -0.651881   
           48005               1854.659654            -99.659654   
           48007                163.409716             -1.409716   
           48009                 19.165098              0.834902   
...                                    ...                   ...   
2020-10-03 48499                632.508069             -0.309701   
           48501                154.616408             13.349910   
           48503               -281.887970             27.606471   
           48505                357.011711             -1.875524   
           48507                438.746788             21.179360   

                      lstm_deaths_predicted  lstm_deaths_residuals  
date       fips_code                                                
2020-08-06 48001                  10.084299              -0.084299  
           48003                   6.054209              -0.054209  
           48005                  37.632364               0.367636  
           48007                  12.128908              -0.128908  
           48009                        NaN                    NaN  
...                                     ...                    ...  
2020-10-03 48499                   3.946016               0.218484  
           48501                   3.913005              -0.036065  
           48503                   6.991850              -0.058177  
           48505                   3.683354              -0.030768  
           48507                  11.097431              -0.018571  

[14750 rows x 4 columns]

In [59]:
lstm_data.to_csv('Database_Files/lstm_daily_data.csv', index=True, date_format='%Y-%m-%d')